## add empty sentences to data

In [ ]:
import json

def add_example_field(json_path, output_path=None):
    # 默认输出路径为原路径加上 "_with_example"
    if output_path is None:
        if json_path.endswith(".json"):
            output_path = json_path.replace(".json", "_t.json")
        else:
            output_path = json_path + "_t.json"

    # 读取原始 JSON 文件
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 为每个词条添加 "例句" 字段
    for entry in data:
        entry["翻译"] = ""

    # 写入新文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"处理完成，已保存到: {output_path}")

# add_example_field("public/data/adj_base.json")
# add_example_field("public/data/adj_steigerung.json")
# add_example_field("public/data/nomen_obj.json")
# add_example_field("public/data/nomen_people.json")
# add_example_field("public/data/verben_base.json")
# add_example_field("public/data/verben_phrasen.json")

处理完成，已保存到: public/data/adj_base_t.json
处理完成，已保存到: public/data/adj_steigerung_t.json


## for adj base

In [ ]:
import json
import os
import re
import random

def build_adj_patterns(word):
    """
    给定一个德语形容词词根，返回所有常见变形的正则（word boundary 保证只匹配单词）。
    如 'gepunktet' → r'\bgepunktet(?:e|es|er|en|em|n|s)?\b'
    """
    # 常见形容词词尾
    endings = ['', 'e', 'es', 'er', 'en', 'em', 'n', 's', 'em', 'er', 'es', 'en']
    patterns = [
        rf"\b{re.escape(word)}{end}\b" for end in endings
    ]
    # 合成为一个大正则
    pattern = re.compile('|'.join(patterns), flags=re.IGNORECASE)
    return pattern

def sentences_adj_base(json_path, opensub_folder):
    # 1. 读取单词本 JSON
    with open(json_path, "r", encoding="utf-8") as f:
        word_list = json.load(f)

    # 2. 构建未被填充例句的词根集合，以及映射（词根 -> json entry 对象引用）
    unfilled = set()
    entry_map = {}
    pattern_map = {}

    for entry in word_list:
        if entry.get("例句", "") == "" and entry.get("单词"):
            word = entry["单词"]
            unfilled.add(word)
            entry_map[word] = entry
            pattern_map[word] = build_adj_patterns(word)

    print(f"初始需要填充例句的单词数: {len(unfilled)}")

    # 3. 遍历 OpenSubtitles 的 txt 文件
    part_files = sorted(
        [f for f in os.listdir(opensub_folder) if f.startswith("part_") and f.endswith(".txt")],
        key=lambda x: int(x.split("_")[1].split(".")[0])
    )
    random.shuffle(part_files)

    for fname in part_files:
        if not unfilled:
            print("所有单词都已找到例句，提前结束！")
            break

        fpath = os.path.join(opensub_folder, fname)
        with open(fpath, "r", encoding="utf-8", errors="ignore") as fin:
            for line in fin:
                sent = line.strip()
                if not sent:
                    continue
                for word in list(unfilled):
                    # 使用正则支持变体匹配
                    if pattern_map[word].search(sent):
                        entry_map[word]["例句"] = sent
                        unfilled.remove(word)
                        print(f"找到例句: {word} → {sent}")
                        break

    print(f"剩余未匹配到的单词数: {len(unfilled)}")

    # 4. 保存新的 JSON
    out_json_path = json_path.replace(".json", "_with_examples.json")
    with open(out_json_path, "w", encoding="utf-8") as fout:
        json.dump(word_list, fout, ensure_ascii=False, indent=2)

    print(f"已完成，保存到 {out_json_path}")

# 示例调用
sentences_adj_base("public/data/adj_base.json", "public/OpenSubtitles")


初始需要填充例句的单词数: 18
找到例句: verheiratet → Wir waren verheiratet.
找到例句: genug → Von den restlichen Sechs hat nur einer eine Anlage, die groß genug ist, um diesen Hubschrauber für die Neulackierung zu verstecken.
找到例句: besonders → (Off) Als besonders ehrbar gelten die Patrizier - also der Stadtadel - und der Klerus.
找到例句: tot → Urin zum Beispiel, als aber auch von toten Menschen, von Leichen.
找到例句: mindestens → Ein Exemplar ist seit mindestens 200 Jahren im Privatbesitz der Familie.
找到例句: dagegen → Und keiner kann etwas dagegen tun.
找到例句: ungefähr → Aimée erahnte ungefähr, was Friedrich so bezauberte, aber sie hatte in Ihrem Leben gelernt, still zu leiden...!
找到例句: männlich → "...habe ich einen Trank, im Mai gekocht, von einer jungen und verspielten männlichen Schwalbe."
找到例句: perfekt → "Sie wissen, sie sind nicht gerade ein perfektes Beispiel mit ihrer Kleidung".
找到例句: weiblich → Weibliche Arbeiterinnen, Lenins Gefolgschaft (bei einem Arbeiterkongress)
找到例句: verwitwet → Die verwitwete Prinze

## for adj steigerung

In [6]:
import json
import os
import re
import random

def build_steigerung_patterns(entry):
    """
    传入一个形容词词条 entry（有 原型/比较级/最高级），返回所有变体的正则pattern。
    """
    # 所有“词干”原型，比较级，最高级
    stems = []
    for key in ["原型", "比较级", "最高级"]:
        v = entry.get(key)
        if v and v not in stems:
            stems.append(v)
    # 常见形容词词尾（规则可自行加/删）
    endings = ['', 'e', 'es', 'er', 'en', 'em', 'n', 's', 'ste', 'sten', 'ster', 'stes', 'ere', 'eren', 'erem', 'eres']
    patterns = []
    for stem in stems:
        patterns += [rf"\b{re.escape(stem)}{end}\b" for end in endings]
    # 合成大正则
    return re.compile('|'.join(patterns), flags=re.IGNORECASE)

def sentences_adj_steigerung(json_path, opensub_folder):
    # 1. 读取单词本 JSON
    with open(json_path, "r", encoding="utf-8") as f:
        word_list = json.load(f)

    # 2. 构建未被填充例句的词根集合，以及映射（词根 -> json entry 对象引用）
    unfilled = set()
    entry_map = {}
    pattern_map = {}

    for entry in word_list:
        if entry.get("例句", "") == "" and entry.get("原型"):
            # 用原型做唯一键
            word = entry["原型"]
            unfilled.add(word)
            entry_map[word] = entry
            pattern_map[word] = build_steigerung_patterns(entry)

    print(f"初始需要填充例句的单词数: {len(unfilled)}")

    # 3. 遍历 OpenSubtitles 的 txt 文件
    part_files = sorted(
        [f for f in os.listdir(opensub_folder) if f.startswith("part_") and f.endswith(".txt")],
        key=lambda x: int(x.split("_")[1].split(".")[0])
    )
    random.shuffle(part_files)

    for fname in part_files:
        if not unfilled:
            print("所有单词都已找到例句，提前结束！")
            break

        fpath = os.path.join(opensub_folder, fname)
        with open(fpath, "r", encoding="utf-8", errors="ignore") as fin:
            for line in fin:
                sent = line.strip()
                if not sent:
                    continue
                for word in list(unfilled):
                    # 使用正则支持全部词干变体匹配
                    if pattern_map[word].search(sent):
                        entry_map[word]["例句"] = sent
                        unfilled.remove(word)
                        print(f"找到例句: {word} → {sent}")
                        break

    print(f"剩余未匹配到的单词数: {len(unfilled)}")

    # 4. 保存新的 JSON
    out_json_path = json_path.replace(".json", "_with_examples.json")
    with open(out_json_path, "w", encoding="utf-8") as fout:
        json.dump(word_list, fout, ensure_ascii=False, indent=2)

    print(f"已完成，保存到 {out_json_path}")

# 示例调用
sentences_adj_steigerung("public/data/adj_steigerung.json", "public/OpenSubtitles")


初始需要填充例句的单词数: 52
找到例句: jung → Er ist zu jung.
找到例句: weit → Und streckt den entgegengesetzten Arm weit vor.
找到例句: alt → Das ist normal in meinem Alter.
找到例句: schön → Schön, dass du mal mitgekommen bist.
找到例句: hoch → Höchstens November.
找到例句: fett → Kaum noch Fett dran.
找到例句: gemein → Das ist gemein.
找到例句: schlimm → Ist es was Schlimmes ?
找到例句: unglaublich → Das ist ja unglaublich.
找到例句: gesund → Er sagt, Sie sind wieder gesund.
找到例句: schwer → Paula hat's schwerer erwischt.
找到例句: eng → Und jetzt enger zusammen.
找到例句: dünn → Dünner Teig, wenig Zucker, viel Obst.
找到例句: heiß → (TV:) Also, was mich betrifft, ich würde den Bären im Freistaat willkommen heißen, wenn er sich entschließen würde, uns zu besuchen."
找到例句: warm → Bei mir wär's wärmer.
找到例句: bereit → In Bayern gilt bereits der Schießbefehl.
找到例句: schick → Meine Wähler schicken mir Drohbriefe.
找到例句: kalt → Also brachten die Franzosen uns in ein Gefängnis in den Pyrenäen, wo Ich mir eine Tuberkulose holte, weil es so kalt war.
找到例句: a

## for noun obj

In [8]:
import json
import os
import re
import random

def strip_article(word):
    # 去掉 der/die/das (注意首字母大写或小写、前后空格)
    return re.sub(r'^(der|die|das|Der|Die|Das)\s+', '', word).strip()

def build_noun_pattern(entry):
    # 取出有效的单数和复数词根（去掉冠词）
    forms = []
    for key in ["单数", "复数"]:
        w = entry.get(key, '').strip()
        if w:
            w = strip_article(w)
            if w and w not in forms:
                forms.append(w)
    # 只匹配整个单词
    patterns = [rf"\b{re.escape(w)}\b" for w in forms]
    return re.compile('|'.join(patterns), flags=re.IGNORECASE)

def sentences_noun(json_path, opensub_folder):
    with open(json_path, "r", encoding="utf-8") as f:
        word_list = json.load(f)

    unfilled = set()
    entry_map = {}
    pattern_map = {}

    for entry in word_list:
        if entry.get("例句", "") == "":
            # 用“单数”去冠词作为唯一key
            key = strip_article(entry.get("单数", "")) or strip_article(entry.get("复数", ""))
            if key:
                unfilled.add(key)
                entry_map[key] = entry
                pattern_map[key] = build_noun_pattern(entry)

    print(f"初始需要填充例句的单词数: {len(unfilled)}")

    part_files = sorted(
        [f for f in os.listdir(opensub_folder) if f.startswith("part_") and f.endswith(".txt")],
        key=lambda x: int(x.split("_")[1].split(".")[0])
    )
    random.shuffle(part_files)

    for fname in part_files:
        if not unfilled:
            print("所有单词都已找到例句，提前结束！")
            break
        fpath = os.path.join(opensub_folder, fname)
        with open(fpath, "r", encoding="utf-8", errors="ignore") as fin:
            for line in fin:
                sent = line.strip()
                if not sent:
                    continue
                for word in list(unfilled):
                    if pattern_map[word].search(sent):
                        entry_map[word]["例句"] = sent
                        unfilled.remove(word)
                        print(f"找到例句: {word} → {sent}")
                        break

    print(f"剩余未匹配到的单词数: {len(unfilled)}")

    out_json_path = json_path.replace(".json", "_with_examples.json")
    with open(out_json_path, "w", encoding="utf-8") as fout:
        json.dump(word_list, fout, ensure_ascii=False, indent=2)

    print(f"已完成，保存到 {out_json_path}")

# 示例调用
sentences_noun("public/data/nomen_obj.json", "public/OpenSubtitles")


初始需要填充例句的单词数: 595
找到例句: Arm → (Pickman) Die blutende Arterie im linken Arm ist das Schlimmste.
找到例句: Nacht → Gute Nacht, Dr. Greene.
找到例句: Minute → Prüfen und EKG in 10 Minuten wiederholen.
找到例句: Wolke → Keine Wolke zu sehen.
找到例句: Morgen → Bist du morgen da?
找到例句: Ball → Dein Ball braucht Luft.
找到例句: Couch → - Du hättest mir die Couch lassen sollen.
找到例句: Heizung → Ich hätte vielleicht die Heizung aufdrehen sollen.
找到例句: Kaffee → Gleich gibt's Kaffee.
找到例句: Loch → Und vorher frisst er dir ein Loch in die Magenschleimhaut.
找到例句: Bad → Geh du schon mal ins Bad.
找到例句: Telefon → (Telefon klingelt)
找到例句: Uhr → War doch richtig, 7:30 Uhr Yoga, nicht?
找到例句: Problem → - Weißt du, er hat Probleme.
找到例句: Arbeit → Susan, John, hier gibt es Arbeit.
找到例句: Jahr → Männlich, 40 Jahre, fiel aus 2 Meter Höhe aufs Gesäß.
找到例句: Name → - Wie ist Ihr Name, Sir?
找到例句: Krankenhaus → - Einem Krankenhaus?
找到例句: Vorsicht → Vorsicht, der ist angenäht, 100 % Rosshaar, gehört mir auch.
找到例句: Auge → - Ich muss Auge

## for noun people

In [11]:
import json
import os
import re
import random

def strip_article(word):
    # 去掉 der/die/das/Das/Die/Der
    return re.sub(r'^(der|die|das|Der|Die|Das)\s+', '', word).strip()

def build_person_pattern(entry):
    # 汇总所有要匹配的形式（单数/复数/男女），都去掉冠词
    fields = ["单数男", "复数男", "单数女", "复数女"]
    forms = []
    for key in fields:
        w = entry.get(key, '').strip()
        if w:
            w = strip_article(w)
            if w and w not in forms:
                forms.append(w)
    patterns = [rf"\b{re.escape(w)}\b" for w in forms]
    if patterns:
        return re.compile('|'.join(patterns), flags=re.IGNORECASE)
    else:
        return None

def sentences_nomen_people(json_path, opensub_folder):
    with open(json_path, "r", encoding="utf-8") as f:
        word_list = json.load(f)

    unfilled = set()
    entry_map = {}
    pattern_map = {}

    for entry in word_list:
        if entry.get("例句", "") == "":
            # 找到第一个非空的字段，去掉冠词做key
            key = None
            for field in ["单数男", "复数男", "单数女", "复数女"]:
                candidate = strip_article(entry.get(field, ""))
                if candidate:
                    key = candidate
                    break
            if key:
                unfilled.add(key)
                entry_map[key] = entry
                pattern = build_person_pattern(entry)
                if pattern:
                    pattern_map[key] = pattern

    print(f"初始需要填充例句的单词数: {len(unfilled)}")

    part_files = sorted(
        [f for f in os.listdir(opensub_folder) if f.startswith("part_") and f.endswith(".txt")],
        key=lambda x: int(x.split("_")[1].split(".")[0])
    )
    random.shuffle(part_files)

    for fname in part_files:
        if not unfilled:
            print("所有单词都已找到例句，提前结束！")
            break
        fpath = os.path.join(opensub_folder, fname)
        with open(fpath, "r", encoding="utf-8", errors="ignore") as fin:
            for line in fin:
                sent = line.strip()
                if not sent:
                    continue
                for word in list(unfilled):
                    if pattern_map[word].search(sent):
                        entry_map[word]["例句"] = sent
                        unfilled.remove(word)
                        print(f"找到例句: {word} → {sent}")
                        break

    print(f"剩余未匹配到的单词数: {len(unfilled)}")

    out_json_path = json_path.replace(".json", "_with_examples.json")
    with open(out_json_path, "w", encoding="utf-8") as fout:
        json.dump(word_list, fout, ensure_ascii=False, indent=2)

    print(f"已完成，保存到 {out_json_path}")

# 示例调用
sentences_nomen_people("public/data/nomen_people.json", "public/OpenSubtitles")


初始需要填充例句的单词数: 63
找到例句: Kind → Kinder?
找到例句: Vater → Sie gibt dauernd mit dem Cadillac von ihrem Vater an.
找到例句: Eltern → Wisst ihr, Mama und Dad, ihr seid wirklich die besten Eltern, die ein Mädchen haben kann.
找到例句: Junge → Tapferes Mädchen, Peg!
找到例句: Sohn → -Wir haben dir unsere Tochter anvertraut.
找到例句: Freund → Maddy ist eine deiner besten Freundinnen.
找到例句: Bruder → Ich habe mit deiner Schwester zu reden.
找到例句: Leute → Amüsiert euch gut, ihr jungen Leute!
找到例句: Kollege → Zusammen mit ihren Kollegen.
找到例句: Designer → Designer:
找到例句: Gast → Und das ist unser Gast.
找到例句: Arzt → - Zum Arzt hören!
找到例句: Mitarbeiter → Im Namen aller Überlebenden der Mitarbeiter des Museums,
找到例句: Nachbar → Hass Nachbar hasse alle, hasse mich!
找到例句: Chef → Sie konnten gerade bestellen, als Chef der Sicherheit.
找到例句: Opa → (Die Woche war fürchterlich, Oma.)
找到例句: Polizist → Nach vielen Jahren ... der Abschluss viele feine Polizisten, der Staat sagt ... kann es sich nicht länger leisten zwei Polizisten Ak

## for verb base

In [9]:
import json
import os
import re
import random

def build_verb_patterns(entry):
    """
    给定一个动词 entry，返回涵盖主流变体的正则 pattern。
    """
    forms = set()
    # 1. 原型
    if entry.get("原型"):
        forms.add(entry["原型"].strip())
    # 2. 现在三单
    if entry.get("现在(第三人称单数)"):
        forms.add(entry["现在(第三人称单数)"].strip())
    # 3. 过去三单
    if entry.get("过去(第三人称单数)"):
        forms.add(entry["过去(第三人称单数)"].strip())
    # 4. 完成分词（如 "hat geholfen"，提取最后一个单词）
    if entry.get("完成"):
        parts = entry["完成"].strip().split()
        if len(parts) >= 2:
            forms.add(parts[-1])
        else:
            forms.add(entry["完成"].strip())
    # 常见动词结尾
    endings = ['', 'e', 'st', 't', 'en', 'te', 'test', 'tet', 'ten', 'et']
    patterns = []
    for f in forms:
        for end in endings:
            patterns.append(rf"\b{re.escape(f)}{end}\b")
    return re.compile('|'.join(patterns), flags=re.IGNORECASE)

def sentences_verben_base(json_path, opensub_folder):
    with open(json_path, "r", encoding="utf-8") as f:
        word_list = json.load(f)

    unfilled = set()
    entry_map = {}
    pattern_map = {}

    for entry in word_list:
        if entry.get("例句", "") == "" and entry.get("原型"):
            word = entry["原型"]
            unfilled.add(word)
            entry_map[word] = entry
            pattern_map[word] = build_verb_patterns(entry)

    print(f"初始需要填充例句的动词数: {len(unfilled)}")

    part_files = sorted(
        [f for f in os.listdir(opensub_folder) if f.startswith("part_") and f.endswith(".txt")],
        key=lambda x: int(x.split("_")[1].split(".")[0])
    )
    random.shuffle(part_files)

    for fname in part_files:
        if not unfilled:
            print("所有动词都已找到例句，提前结束！")
            break

        fpath = os.path.join(opensub_folder, fname)
        with open(fpath, "r", encoding="utf-8", errors="ignore") as fin:
            for line in fin:
                sent = line.strip()
                if not sent:
                    continue
                for word in list(unfilled):
                    if pattern_map[word].search(sent):
                        entry_map[word]["例句"] = sent
                        unfilled.remove(word)
                        print(f"找到例句: {word} → {sent}")
                        break

    print(f"剩余未匹配到的动词数: {len(unfilled)}")

    out_json_path = json_path.replace(".json", "_with_examples.json")
    with open(out_json_path, "w", encoding="utf-8") as fout:
        json.dump(word_list, fout, ensure_ascii=False, indent=2)

    print(f"已完成，保存到 {out_json_path}")

# 示例调用
sentences_verben_base("public/data/verben_base.json", "public/OpenSubtitles")


初始需要填充例句的动词数: 187
找到例句: sehen → Sie sehen blass aus.
找到例句: werden → Sie sahen morgens schon schlecht aus und es wurde schlimmer.
找到例句: gehen → Es geht mir gut.
找到例句: machen → Nein, ich habe heute früher Schluss gemacht.
找到例句: essen → Hast du gegessen?
找到例句: nehmen → Ich habe schon was genommen.
找到例句: denken → Ich habe mir schon gedacht, dass du früher kommst.
找到例句: schlafen → Ich will nur schlafen.
找到例句: bleiben → Du musst jetzt stark bleiben.
找到例句: schreiben → Warum schreiben Sie mir?
找到例句: kommen → Sonst kommen Sie zu mir.
找到例句: dauern → -Wenn es nicht zu lang dauert.
找到例句: sagen → Er sagte, es ginge ihm gut, aber er sah heute gar nicht gut aus.
找到例句: haben → Nach dem Zoff in der Bar hat mein Bruder alles bezahlt.
找到例句: mögen → Ich mag dich mehr
找到例句: sitzen → Was sitzt du zu Hause.
找到例句: brauchen → Nach einer Trennung braucht man Süßes.
找到例句: bringen → Er brachte sie mit einem Zug zum Schweigen.
找到例句: geben → Geben Sie sich nicht die Schuld.
找到例句: leben → Sie mischt sich in sein Leb

In [ ]:
# add the sentence for verben phrasen manually